In [1]:
from sys import platform
import pandas as pd
import matplotlib.pyplot as plt
import palettable
import sqlite3

from scipy.optimize import curve_fit
import numpy as np

In [2]:
def func2(x,a,b):
    return a+b/x

In [3]:
## deflated method done with 100 pure wstat eigenpotentials
## 这里的kin100出自wstat kin-100-400的计算
kin100 = [[  -9.553140,     -9.552188,     -7.553253,     -7.554302,     -7.554744,     -7.553960,
             -7.553888,     -7.503336,     -7.503982,     -7.503983,     -7.504368,     -6.284428,
             -6.284492,     -6.286003,     -6.283752,     -6.285335,     -1.515139,     -1.513558,
             -1.515885,     -0.369182,     -0.371171,     -0.369485,      0.443791,      0.448265,
              0.444938,      0.656004],
          [  -9.593658,     -9.593395,     -7.659079,     -7.659810,     -7.660296,     -7.659424,
             -7.659554,     -7.600693,     -7.601482,     -7.600980,     -7.600923,     -6.405959,
             -6.406169,     -6.407974,     -6.406099,     -6.406750,     -1.690335,     -1.688204,
             -1.689951,     -0.641875,     -0.638458,     -0.641853,      0.278874,      0.283493,
              0.280169,      0.423163],
          [  -9.644994,     -9.645610,     -7.704981,     -7.705857,     -7.706205,     -7.705728,
             -7.705620,     -7.650199,     -7.650805,     -7.650511,     -7.650312,     -6.466867,
             -6.467541,     -6.469040,     -6.467699,     -6.468505,     -1.775838,     -1.773430,
             -1.775073,     -0.700471,     -0.697346,     -0.700091,      0.195752,      0.200300,
              0.197334,      0.363857],
          [  -9.702559,     -9.702841,     -7.749545,     -7.749316,     -7.750018,     -7.749606,
             -7.749450,     -7.687958,     -7.688518,     -7.688502,     -7.688204,     -6.522220,
             -6.522858,     -6.524691,     -6.523354,     -6.524134,     -1.844806,     -1.842206,
             -1.843913,     -0.774824,     -0.771182,     -0.773901,      0.133742,      0.138497,
              0.135286,      0.301443]
         ]

## deflated method done with 200 pure wstat eigenpotentials
## 这里的kin200出自wstat kin-200-400的计算
kin200 = [[  -9.562194,     -9.563186,     -7.640897,     -7.641318,     -7.641666,     -7.640971,
             -7.641063,     -7.577189,     -7.577503,     -7.577596,     -7.577347,     -6.383491,
             -6.383401,     -6.385801,     -6.384213,     -6.384379,     -1.728224,     -1.725993,
             -1.727686,     -0.675610,     -0.679602,     -0.675547,      0.242558,      0.247321,
              0.243958,      0.408129],
          [  -9.621159,     -9.617422,     -7.674203,     -7.674377,     -7.675042,     -7.674342,
             -7.674245,     -7.622265,     -7.622583,     -7.622582,     -7.622310,     -6.436009,
             -6.436695,     -6.438448,     -6.436978,     -6.437657,     -1.810541,     -1.807932,
             -1.809128,     -0.738628,     -0.735460,     -0.737795,      0.156854,      0.160776,
              0.158329,      0.321858],
          [  -9.665587,     -9.665988,     -7.720873,     -7.721241,     -7.721693,     -7.720971,
             -7.721249,     -7.670765,     -7.671149,     -7.671087,     -7.670877,     -6.491884,
             -6.492501,     -6.493930,     -6.492936,     -6.493335,     -1.872391,     -1.870095,
             -1.871180,     -0.802374,     -0.799458,     -0.801968,      0.105794,      0.106166,
              0.107340,      0.296921],
          [  -9.711451,     -9.711489,     -7.753478,     -7.753658,     -7.754168,     -7.753440,
             -7.753777,     -7.690812,     -7.691170,     -7.691113,     -7.690963,     -6.528265,
             -6.529046,     -6.530634,     -6.529335,     -6.530016,     -1.931490,     -1.929250,
             -1.930178,     -0.845789,     -0.842882,     -0.845297,      0.054108,      0.054743,
              0.053263,      0.242693]
         ]

ws =  [## pure wstat with 100 eigenpotentials
       [ -9.450455,     -9.449117,     -7.375195,     -7.376504,     -7.376760,     -7.375695,
         -7.375320,     -7.330542,     -7.330930,     -7.330444,     -7.330198,     -6.055938,
         -6.058203,     -6.060283,     -6.057795,     -6.058553,     -1.219844,     -1.217021,
         -1.219063,     -0.202848,     -0.201328,     -0.203251,      0.774942,      0.778867,
          0.776537,      0.849116],
       ## pure wstat with 200 eigenpotentials
       [ -9.526203,     -9.526404,     -7.552343,     -7.553910,     -7.553224,     -7.553611,
         -7.554594,     -7.501964,     -7.502599,     -7.502416,     -7.501773,     -6.278196,
         -6.278496,     -6.280512,     -6.279170,     -6.280402,     -1.541886,     -1.540786,
         -1.539174,     -0.406902,     -0.404883,     -0.408380,      0.426846,      0.432148,
          0.429462,      0.629144],   
       ## pure wstat with 1000 eigenpotentials
       [ -9.981530,     -9.981091,     -7.855299,     -7.855616,     -7.854009,     -7.855274,
         -7.855667,     -7.794270,     -7.794598,     -7.794509,     -7.794297,     -6.647865,
         -6.648557,     -6.650279,     -6.648845,     -6.649465,     -2.071820,     -2.069624,
         -2.070682,     -0.975228,     -0.972433,     -0.974728,     -0.076066,     -0.071409,
         -0.074446,      0.055310],
       # pure wstat with 2000 eigenpotentials
       [-10.079247,    -10.078958,     -7.922906,     -7.918295,     -7.920451,     -7.920446,
         -7.914163,     -7.871858,     -7.872162,     -7.872100,     -7.871882,     -6.748516,
         -6.749128,     -6.750831,     -6.749397,     -6.750039,     -2.191772,     -2.189595,
         -2.190646,     -1.078972,     -1.076225,     -1.078489,     -0.188615,     -0.183972,
         -0.187000,     -0.034661],
       ## pure wstat with 3000 eigenpotentials
       [ -10.110241,    -10.109804,     -7.941409,     -7.941603,     -7.938950,     -7.941527,
         -7.941873,     -7.896365,     -7.896629,     -7.896560,     -7.896354,     -6.779231,
         -6.780401,     -6.781603,     -6.780486,     -6.781100,     -2.225420,     -2.223285,
         -2.224046,     -1.113091,     -1.110357,     -1.112424,     -0.222025,     -0.217333,
         -0.220464,     -0.068442]
]

In [4]:
from scipy.optimize import curve_fit

num1 = range(200,501,100)
num2 = range(300,601,100)
num3 = [100, 200, 1000, 2000,3000]

kin100_fit = []
kin200_fit = []
ws_fit  = []
for st in range(26):
    propt, propv = curve_fit(func2, num1, [kin100[j][st] for j in range(len(kin100))])
    kin100_fit.append(propt[0]-0.65)
for st in range(26):    
    propt, propv = curve_fit(func2, num2, [kin200[j][st]  for j in range(len(kin200))])
    kin200_fit.append(propt[0]-0.65)
for st in range(26):    
    propt, propv = curve_fit(func2, num3, [    ws[j][st]  for j in range(len(ws))])
    ws_fit.append(propt[0]-0.65)
    
for i in range(len(ws_fit)):
        print("Level index:%5d, Fit100: %6.2f, Fit200: %6.2f, ref:%6.2f"%(i+105, kin100_fit[i],kin200_fit[i],ws_fit[i]))

Level index:  105, Fit100: -10.42, Fit200: -10.50, ref:-10.72
Level index:  106, Fit100: -10.42, Fit200: -10.50, ref:-10.72
Level index:  107, Fit100:  -8.52, Fit200:  -8.51, ref: -8.58
Level index:  108, Fit100:  -8.52, Fit200:  -8.51, ref: -8.58
Level index:  109, Fit100:  -8.52, Fit200:  -8.51, ref: -8.58
Level index:  110, Fit100:  -8.52, Fit200:  -8.51, ref: -8.58
Level index:  111, Fit100:  -8.52, Fit200:  -8.51, ref: -8.58
Level index:  112, Fit100:  -8.46, Fit200:  -8.46, ref: -8.53
Level index:  113, Fit100:  -8.46, Fit200:  -8.46, ref: -8.53
Level index:  114, Fit100:  -8.46, Fit200:  -8.46, ref: -8.53
Level index:  115, Fit100:  -8.45, Fit200:  -8.46, ref: -8.53
Level index:  116, Fit100:  -7.32, Fit200:  -7.32, ref: -7.40
Level index:  117, Fit100:  -7.32, Fit200:  -7.32, ref: -7.40
Level index:  118, Fit100:  -7.32, Fit200:  -7.32, ref: -7.41
Level index:  119, Fit100:  -7.32, Fit200:  -7.32, ref: -7.40
Level index:  120, Fit100:  -7.32, Fit200:  -7.32, ref: -7.41
Level in